## Importe

In [ ]:
import cv2
import numpy as np
import os

## Bild Laden

In [2]:
class Colors:
    @staticmethod
    def red():
        return (239, 8, 8)

    @staticmethod
    def darkred():
        return (148, 20, 33)

    @staticmethod
    def orange():
        return (255, 134, 57)

    @staticmethod
    def darkorange():
        return (181, 101, 24)

    @staticmethod
    def darkgold():
        return (165, 150, 0)

    @staticmethod
    def yellow():
        return (255, 251, 0)

    @staticmethod
    def lime():
        return (156, 219, 0)

    @staticmethod
    def green():
        return (66, 190, 66)

    @staticmethod
    def darkgreen():
        return (41, 138, 82)

    @staticmethod
    def teal():
        return (41, 77, 74)

    @staticmethod
    def darkteal():
        return (41, 77, 74)

    @staticmethod
    def blue():
        return (66, 109, 239)

    @staticmethod
    def darkblue():
        return (41, 40, 231)

    @staticmethod
    def purple():
        return (90, 0, 165)

    @staticmethod
    def violet():
        return (148, 97, 255)

    @staticmethod
    def turkis():
        return (99, 255, 206)

    @staticmethod
    def white():
        return (255, 255, 255)

    @staticmethod
    def gray():
        return (99, 109, 123)

    @staticmethod
    def magenta():
        return (255, 138, 255)

    @staticmethod
    def pink():
        return (255, 48, 255)

### Beliebige Farbe extrahieren

### Alle Farben einzeln extrahieren + löschen was zu wohl nicht im spiel vorkommt

In [3]:

def player_color(picture, threshold=20,min_contour_area=5,output_path="output.png",result_bgr=None):
    
    gray = cv2.cvtColor(picture, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    shape_count = 0
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_contour_area:
            shape_count += 1
            cv2.drawContours(picture, [contour], -1, (0, 255, 0), 2)
    output_path = output_path.replace(".png", f"_{shape_count}.png")
    if shape_count > 0:
        cv2.imwrite(output_path, result_bgr)
    

def filter_red_pixels_cv2(input_path, output_path, target_color, tolerance=50):
    """
    OpenCV Version für rote Pixel Filterung
    """
    # Bild laden
    img = cv2.imread(input_path)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
    target_r, target_g, target_b = target_color
    
    color_mask = (
        (np.abs(img[:, :, 0] - target_r) < tolerance) &  # Rot
        (np.abs(img[:, :, 1] - target_g) < tolerance) &  # Grün
        (np.abs(img[:, :, 2] - target_b) < tolerance)    # Blau
    )
    
    # Ergebnis erstellen
    result = np.zeros_like(img)
    result[color_mask] = img[color_mask]
    
    # Speichern
    result_bgr = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
    #player_color(result, threshold=20,min_contour_area=500,output_path=output_path,result_bgr=result_bgr)
    cv2.imwrite(output_path, result_bgr)
    
    print(f"OpenCV: Bild gespeichert: {output_path}")


In [4]:
colors = Colors()

for func_name in dir(colors):
    if func_name.startswith("__"):
        continue
    func = getattr(colors, func_name)
    filter_red_pixels_cv2(r"E:\Coding\Python\MachineLearning_SC\notebooks\pictures\screenshot_20250911_025258.png", f"pictures/pic_{func_name}.png",
                      target_color=func(), tolerance=40)

OpenCV: Bild gespeichert: pictures/pic_blue.png
OpenCV: Bild gespeichert: pictures/pic_darkblue.png
OpenCV: Bild gespeichert: pictures/pic_darkgold.png
OpenCV: Bild gespeichert: pictures/pic_darkgreen.png
OpenCV: Bild gespeichert: pictures/pic_darkorange.png
OpenCV: Bild gespeichert: pictures/pic_darkred.png
OpenCV: Bild gespeichert: pictures/pic_darkteal.png
OpenCV: Bild gespeichert: pictures/pic_gray.png
OpenCV: Bild gespeichert: pictures/pic_green.png
OpenCV: Bild gespeichert: pictures/pic_lime.png
OpenCV: Bild gespeichert: pictures/pic_magenta.png
OpenCV: Bild gespeichert: pictures/pic_orange.png
OpenCV: Bild gespeichert: pictures/pic_pink.png
OpenCV: Bild gespeichert: pictures/pic_purple.png
OpenCV: Bild gespeichert: pictures/pic_red.png
OpenCV: Bild gespeichert: pictures/pic_teal.png
OpenCV: Bild gespeichert: pictures/pic_turkis.png
OpenCV: Bild gespeichert: pictures/pic_violet.png
OpenCV: Bild gespeichert: pictures/pic_white.png
OpenCV: Bild gespeichert: pictures/pic_yellow.png


In [5]:
def sum_colors(output_path):
    """
    Summiert die RGB-Werte jedes Pixels in einem Bild und speichert das Ergebnis.
    """
    
    # Alle Dateien im Ordner durchlaufen
    files = []
    for file_name in os.listdir(output_path):
        if file_name.startswith("pic") and file_name.endswith(('.png', '.jpg', '.jpeg')):
            final_path = os.path.join(output_path, file_name)
            img = cv2.imread(final_path)
            files.append(img)
            
    
    
    final = np.zeros_like(files[0])
    for i in range(len(files)):        
        final = cv2.add(final, files[i])        
            
    cv2.imwrite("pictures/sum_colors.png", final)
            

In [6]:
sum_colors(r"pictures")

### Konturen erkennen und filtern

In [19]:
import cv2
import numpy as np

img = cv2.imread('pictures/sum_colors.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Schwellenwert anwenden, um ein Binärbild zu erhalten (Spieler vs. Hintergrund)
# Der Schwellenwert (z.B. 50) muss eventuell angepasst werden!
_, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

output_img = np.zeros_like(img)

for contour in contours:
    
    area = cv2.contourArea(contour)

    if 10 < area < 5000: # erkannte kontur größe -> anpassen
        cv2.drawContours(output_img, [contour], -1, (255, 255, 255), cv2.FILLED)
        bounding_box = cv2.boundingRect(contour)
        x, y, w, h = bounding_box
        output_img[y:y+h, x:x+w] = img[y:y+h, x:x+w] 
        

cv2.imwrite("pictures/konturen.png", output_img)

True

### Morphologische Operationen

In [23]:
import cv2
import numpy as np

img = cv2.imread('pictures/sum_colors.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 10, 50, cv2.THRESH_BINARY) # Wieder Binärbild

# Erosion: Macht helle Bereiche kleiner, entfernt kleine Rauschen
kernel = np.ones((2,2),np.uint8) # Größe des Kernels anpassen
eroded = cv2.erode(thresh, kernel, iterations = 1)

# Dilatation: Macht helle Bereiche größer (kann nützlich sein, um Spieler wieder zu verbinden)
dilated = cv2.dilate(eroded, kernel, iterations = 1)

# Opening (Erosion gefolgt von Dilatation) ist gut, um kleine Objekte zu entfernen
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# Und dann das Ergebnis wieder auf das Originalbild anwenden
res = cv2.bitwise_and(img, img, mask=opening)


cv2.imwrite("pictures/morph.png", res)

True

### Bild abziehen
- Start Bild als referenz nehmen und von allen kommenden alle identischen Pixel löschen -> damit auch spielfarben ermitteln -> komplett automatisierbar

In [23]:
pfad = r"E:\Coding\Python\MachineLearning_SC\data\raw\screenshots\Session_5s_2025-09-11_02-44-10"

start = r"\screenshot_20250911_024441.png"

test = r"\screenshot_20250911_025635.png"

start_pfad = pfad + start
test_pfad = pfad + test

img1 = cv2.imread(start_pfad)
img2 = cv2.imread(test_pfad)

diff = cv2.absdiff(img1, img2)

gray_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray_diff, 20, 255, cv2.THRESH_BINARY)

# Wende die Maske auf Bild 2 an, um nur die unterschiedlichen Pixel zu behalten
result = cv2.bitwise_and(img2, img2, mask=mask)
cv2.imwrite("pictures/subtract.png", result)


True